In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [2]:
#Preprocessing the dataset images
from keras.preprocessing.image import ImageDataGenerator

Imagegen = ImageDataGenerator(rotation_range = 30,
                              width_shift_range = 0.2, 
                              height_shift_range = 0.2,
                              rescale = 1/255,
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              fill_mode = 'nearest')

train = Imagegen.flow_from_directory('face-mask-dataset/train', target_size = (150,150), batch_size = 20, class_mode = 'binary')
test = Imagegen.flow_from_directory('face-mask-dataset/test', target_size = (150,150), batch_size = 20, class_mode = 'binary')

Using TensorFlow backend.


Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [3]:
train.class_indices

{'with_mask': 0, 'without_mask': 1}

In [4]:
#Building the model
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout

model = Sequential([Conv2D(filters = 32, kernel_size = (3,3), input_shape = (150,150,3), activation = 'relu'),
                   MaxPooling2D((2,2)),
                   Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'),
                   MaxPooling2D((2,2)),
                   Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'),
                   MaxPooling2D((2,2)),
                   Flatten(),
                   Dense(128, activation = 'relu'),
                   Dense(1, activation = 'sigmoid')])

In [5]:
#Compiling the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [6]:
#Training the model
model.fit_generator(train, epochs = 10, validation_data = test)

Epoch 1/10
66/66 [==============================] - 24s 359ms/step - loss: 0.5145 - acc: 0.7240 - val_loss: 0.2634 - val_acc: 0.8711
Epoch 2/10
66/66 [==============================] - 27s 409ms/step - loss: 0.3035 - acc: 0.8812 - val_loss: 0.1896 - val_acc: 0.9330
Epoch 3/10
66/66 [==============================] - 28s 422ms/step - loss: 0.2454 - acc: 0.9047 - val_loss: 0.2089 - val_acc: 0.9278
Epoch 4/10
66/66 [==============================] - 27s 406ms/step - loss: 0.2264 - acc: 0.9091 - val_loss: 0.1141 - val_acc: 0.9691
Epoch 5/10
66/66 [==============================] - 27s 409ms/step - loss: 0.1660 - acc: 0.9444 - val_loss: 0.1125 - val_acc: 0.9588
Epoch 6/10
66/66 [==============================] - 27s 404ms/step - loss: 0.1846 - acc: 0.9342 - val_loss: 0.1663 - val_acc: 0.9433
Epoch 7/10
66/66 [==============================] - 27s 409ms/step - loss: 0.1456 - acc: 0.9436 - val_loss: 0.0985 - val_acc: 0.9691
Epoch 8/10
66/66 [==============================] - 27s 411ms/step - 

In [7]:
model.save('Face_mask_detection.h5')

In [2]:
#Loading the model

from keras.models import load_model
model = load_model('Face_mask_detection.h5')

Using TensorFlow backend.


In [5]:
#Real time face mask detection

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    cv2.imwrite('test.jpg', frame)
    
    #face detection
    detector = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
    face = detector.detectMultiScale(frame)
    
    from keras.preprocessing import image
    
    test_img = image.load_img('test.jpg', target_size = (150,150))
    test_img = image.img_to_array(test_img)
    test_img = np.expand_dims(test_img, axis = 0)
    test_img = test_img/255
    
    if model.predict_classes(test_img)[0][0] == 1:
        for x, y, w, h in face:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 3)
            cv2.putText(frame, 'Please put on a mask', (10,20), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    else:
        for x, y, w, h in face:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0,), 3)
            cv2.putText(frame, 'You are safe', (10,20), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
    cv2.imshow('Face_mask_detector', frame)
    
    k = cv2.waitKey(10)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>